# Hello World
pr107 is the only problem so far for those <= 400

In [1]:
import tsplib95
from pathlib import Path
import itertools
import numpy as np
from metric_split import balanced_metric_split
import threading
from concurrent.futures import ThreadPoolExecutor

TSP_LIB_PATH = Path("../../ALL_tsp")
DIMENSION_THRESHOLD = 200

In [2]:
all_problems : list[tsplib95.models.StandardProblem] = []

for file in sorted(TSP_LIB_PATH.iterdir()): # Loop through every tsp
    if file.suffix != ".tsp" or not file.is_file():
        continue
    problem = tsplib95.load(f"{file.absolute()}")
    if problem.edge_weight_type in ["EXPLICIT"]:
        continue # TODO: include ATT and GEO and maybe EUC_3D
    
    all_problems.append(problem)
    print(f"Added {problem.name}")

print("Found", len(all_problems), "euclidean TSPs")

Added Tnm100.tsp
Added Tnm103.tsp
Added Tnm106.tsp
Added Tnm109.tsp
Added Tnm112.tsp
Added Tnm115.tsp
Added Tnm118.tsp
Added Tnm121.tsp
Added Tnm124.tsp
Added Tnm127.tsp
Added Tnm130.tsp
Added Tnm133.tsp
Added Tnm136.tsp
Added Tnm139.tsp
Added Tnm142.tsp
Added Tnm145.tsp
Added Tnm148.tsp
Added Tnm151.tsp
Added Tnm154.tsp
Added Tnm157.tsp
Added Tnm160.tsp
Added Tnm163.tsp
Added Tnm166.tsp
Added Tnm169.tsp
Added Tnm172.tsp
Added Tnm175.tsp
Added Tnm178.tsp
Added Tnm181.tsp
Added Tnm184.tsp
Added Tnm187.tsp
Added Tnm190.tsp
Added Tnm193.tsp
Added Tnm196.tsp
Added Tnm199.tsp
Added Tnm52.tsp
Added Tnm55.tsp
Added Tnm58.tsp
Added Tnm61.tsp
Added Tnm64.tsp
Added Tnm67.tsp
Added Tnm70.tsp
Added Tnm73.tsp
Added Tnm76.tsp
Added Tnm79.tsp
Added Tnm82.tsp
Added Tnm85.tsp
Added Tnm88.tsp
Added Tnm91.tsp
Added Tnm94.tsp
Added Tnm97.tsp
Added a280
Added ali535
Added ara238025
Added att48
Added att532
Added bby34656
Added bbz25234
Added bch2762
Added bck2217
Added bcl380
Added beg3293
Added berlin52
A

In [9]:
# TEST
def geo_based_dist_matrix(nodes_array, radius: float = 6378.388) -> np.ndarray:
    lat = np.deg2rad(nodes_array[:, 0])
    lon = np.deg2rad(nodes_array[:, 1])

    # Broadcast to pairwise differences
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]

    q1 = np.cos(lon_i - lon_j)
    q2 = np.cos(lat_i - lat_j)
    q3 = np.cos(lat_i + lat_j)

    return radius * np.arccos(0.5 * ((1 + q1) * q2 - (1 - q1) * q3))

def convert_to_dist_matrix(problem: tsplib95.models.StandardProblem) -> np.ndarray:
    """Convert a tsplib95 StandardProblem to a distance matrix."""
    n = int(problem.dimension)  # Cast to int to resolve type mismatch
    dist_matrix = np.zeros((n, n))
    nodes_array = np.array(list(problem.node_coords.values()))
    if problem.edge_weight_type in ["EUC_2D", "CEIL_2D", "ATT"]:
        dist_matrix = np.linalg.norm(nodes_array[:, np.newaxis] - nodes_array[np.newaxis, :], axis=2)
        if problem.edge_weight_type == "ATT":
            dist_matrix /= np.sqrt(10)
    elif "GEO" in problem.edge_weight_type:
        dist_matrix = geo_based_dist_matrix(nodes_array)
    else:
        raise ValueError(f"Unsupported edge weight type: {problem.edge_weight_type}")
    if "CEIL" in problem.edge_weight_type:
        dist_matrix = np.ceil(dist_matrix)
    return dist_matrix


all_dist_matrices = ((problem.name, convert_to_dist_matrix(problem)) for problem in all_problems if 2 * DIMENSION_THRESHOLD < int(problem.dimension) <= 4 * DIMENSION_THRESHOLD)

In [10]:
forest_amounts: dict[str, tuple[int, int] | None] = {}

def _compute_split(args: tuple[str, np.ndarray]):
    name, dist_matrix = args
    print(name, dist_matrix.shape[0])
    forest = balanced_metric_split(dist_matrix, s=1.0, k=3)
    return name, None if len(forest) == 1 else tuple(len(cluster) for cluster in forest)
    
# Run splits concurrently; order preserved with executor.map
with ThreadPoolExecutor() as executor:
    for name, result in executor.map(_compute_split, all_dist_matrices):
        forest_amounts[name] = result

ali535 535
att532 532
d493 493
d657 657
fl417 417
gr431 431
gr666 666
p654 654
pbk411 411
pbm436 436
pbn423 423
pcb442 442
pr439 439
rat575 575
rat783 783
rbu737 737
rbx711 711
u574 574
u724 724
uy734 734
xql662 662


: 

: 

In [8]:
forest_amounts

{'a280': None,
 'bcl380': None,
 'gil262': None,
 'gr202': None,
 'gr229': None,
 'lin318': None,
 'linhp318': None,
 'pbl395': None,
 'pka379': None,
 'pma343': None,
 'pr226': None,
 'pr264': None,
 'pr299': None,
 'rd400': None,
 'ts225': None,
 'tsp225': None,
 'xqg237': None}